## Algoritmo genético utilizando DEAP

deap https://deap.readthedocs.io/en/master/

The are a few python modules that are built specially for GA development. DEAP is one of them

"DEAP is a novel evolutionary computation framework for rapid prototyping and testing of ideas. It seeks to make algorithms explicit and data structures transparent. It works in perfect harmony with parallelisation mechanism such as multiprocessing and SCOOP"

### Libraries

In [8]:
import array
import random
import json

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

### Data Import
We are going to use presolved instance of the TSP (17,24 and 120 nodes). These presolved instances will help to see how good the GA performed when compared to the optimal solution.

In [9]:
with open("/home/storreglosa/Projects/03_Metaheuristicas_transporte/Genetic_Algoritm_Vehicle_Routing/sample_data/gr120.json", "r") as tsp_data:
    tsp = json.load(tsp_data)

distance_map = tsp["DistanceMatrix"]

IND_SIZE = tsp["TourSize"]

### DEAP Genetic algoritms library
DEAP comes with a lot of predefined operators and algorithms, these can be used to solve a variety of problems, and we can also create custom operators for different problems, but here the ones that are provided by the module will be enought.

In [ ]:

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)

toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalTSP(individual):
    distance = distance_map[individual[-1]][individual[0]]
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += distance_map[gene1][gene2]
    return distance,

toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=30)
toolbox.register("evaluate", evalTSP)

def main():
    random.seed(169)

    pop = toolbox.population(n=1000)

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 500, stats=stats, 
                        halloffame=hof)
    
    return pop, stats, hof

if __name__ == "__main__":
    pop,stats,hof=main()
    print(hof)
    print(evalTSP(hof[0]))